# Multi-Teacher On-Policy Distillation

This notebook demonstrates **multi-teacher on-policy distillation (OPD)**, an extension of standard on-policy distillation where a student model learns from **multiple teacher models** simultaneously.

## Key Concepts

Unlike single-teacher distillation, multi-teacher distillation allows:
- **Diverse knowledge sources**: Each teacher may excel at different aspects of the task
- **Ensemble benefits**: Combining multiple teachers can produce better guidance than any single teacher
- **Specialized teachers**: Different teachers can be trained on different datasets or with different objectives

In this notebook, you'll:
1. Train a small student model (Qwen3-0.6B) using two teacher models
2. Teacher 1: Qwen3-4B-Instruct-2507 (strong reasoning)
3. Teacher 2: Qwen3-3B-Instruct (faster, different training)
4. Learn on [GSM8K](https://huggingface.co/datasets/openai/gsm8k) math problems

## Requirements

You'll need an **A100 GPU (40 GB)** or better. With smaller GPUs, reduce `samples_per_prompt` and `max_new_tokens`.

## References

- Blog: [Thinking Machines - On-Policy Distillation](https://thinkingmachines.ai/blog/on-policy-distillation/)
- Paper: [Agarwal et al. 2024](https://arxiv.org/abs/2306.13649)
- Code: [tinker-cookbook multi-teacher implementation](https://github.com/thinking-machines-lab/tinker-cookbook/blob/main/tinker_cookbook/recipes/distillation/on_policy_multi_teacher.py)

In [ ]:
#@title 🛠️ Setup
!nvidia-smi -L || true

import os, sys, random, numpy as np, torch, json, time, platform, math
print("Python:", sys.version)
print("CUDA available:", torch.cuda.is_available())

# Qwen3 requires Transformers >= 4.51
try:
    get_ipython().run_line_magic("uv", "pip -q install transformers==4.51.3 accelerate==1.4.0 peft==0.14.0 datasets==3.3.2 evaluate==0.4.3 sentencepiece protobuf tqdm matplotlib > /dev/null")
except Exception:
    get_ipython().run_line_magic("pip", "-q install transformers==4.51.3 accelerate==1.4.0 peft==0.14.0 datasets==3.3.2 evaluate==0.4.3 sentencepiece protobuf tqdm matplotlib > /dev/null")

import transformers, datasets, peft, accelerate, matplotlib
print("Transformers:", transformers.__version__)
print("Accelerate:", accelerate.__version__)
print("PEFT:", peft.__version__)
print("matplotlib:", matplotlib.__version__)

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
assert DEVICE == "cuda", "Please connect a GPU (A100+ recommended)."

def print_header():
    print("== Environment ==")
    print(dict(
        python=sys.version,
        torch=torch.__version__,
        transformers=transformers.__version__,
        accelerate=accelerate.__version__,
        peft=peft.__version__,
        cuda=torch.version.cuda if torch.cuda.is_available() else "cpu",
        device_name=torch.cuda.get_device_name(0) if torch.cuda.is_available() else "cpu",
        platform=platform.platform(),
        seed=SEED
    ))
print_header()

In [ ]:
import os, sys, time, json, random, platform
from dataclasses import dataclass
from typing import Optional, List, Dict, Tuple

import numpy as np
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm

import pandas as pd
from IPython.display import display

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
import re

# --------------------------
# Reproducibility & device
# --------------------------
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
assert DEVICE == "cuda", "A CUDA GPU is required."

## Configuration

The key difference from single-teacher OPD is that we now specify **multiple teachers**. Each teacher contributes to the distillation with a configurable weight.

### Teacher Weighting Strategies

1. **Uniform**: Equal weight to all teachers (default)
2. **Performance-based**: Weight by baseline accuracy
3. **Dynamic**: Adjust weights during training based on agreement

For this demo, we use **uniform weighting** for simplicity.

In [ ]:
# --------------------------
# Multi-Teacher Config
# --------------------------
@dataclass
class TeacherConfig:
    """Configuration for a single teacher model."""
    model_id: str
    weight: float = 1.0  # Contribution weight (will be normalized)
    description: str = ""

@dataclass
class Config:
    # Student model
    student_id: str = "Qwen/Qwen3-0.6B-Base"
    
    # Multiple teachers with their configurations
    teachers: List[TeacherConfig] = None
    
    def __post_init__(self):
        if self.teachers is None:
            # Default: two teachers with equal weights
            self.teachers = [
                TeacherConfig(
                    model_id="Qwen/Qwen3-4B-Instruct-2507",
                    weight=1.0,
                    description="Strong reasoning, latest version"
                ),
                TeacherConfig(
                    model_id="Qwen/Qwen3-3B-Instruct",
                    weight=1.0,
                    description="Faster, different training distribution"
                ),
            ]
        # Normalize weights
        total_weight = sum(t.weight for t in self.teachers)
        for t in self.teachers:
            t.weight = t.weight / total_weight

    # Prompting
    prompt_template: str = (
        "Solve step by step.\n"
        "Give ONLY ONE final numeric answer (no units), inside square brackets.\n"
        "Problem: {question}\n\nSolution:"
    )
    max_new_tokens: int = 256

    # Generation temps
    eval_temperature: float = 0.0   # greedy for eval
    train_temperature: float = 0.7  # sampling for on-policy data

    # Training schedule
    steps: int = 50
    batch_prompts: int = 4
    samples_per_prompt: int = 4
    lr: float = 1e-4
    weight_decay: float = 0.0
    grad_accum: int = 1

    # Micro-batching
    student_mb: int = 8

    # Monitoring
    log_every: int = 10
    val_every: int = 10
    val_sample_n: int = 100
    ema_momentum: float = 0.9

    # Validation size
    val_rows: Optional[int] = None

    # Output dir
    run_root: str = f"./run_multi_teacher_{int(time.time())}"

cfg = Config()
os.makedirs(cfg.run_root, exist_ok=True)

print("\n=== Multi-Teacher Configuration ===")
print(f"Student: {cfg.student_id}")
print(f"\nTeachers ({len(cfg.teachers)}):")
for i, t in enumerate(cfg.teachers, 1):
    print(f"  {i}. {t.model_id}")
    print(f"     Weight: {t.weight:.3f}")
    print(f"     Description: {t.description}")
print(f"\nTraining steps: {cfg.steps}")
print(f"Batch size: {cfg.batch_prompts} prompts × {cfg.samples_per_prompt} samples")

## Dataset: GSM8K

We use the same dataset and parsing utilities as single-teacher OPD.

In [ ]:
# --------------------------
# Data: GSM8K
# --------------------------
def render_prompt(question: str) -> str:
    return cfg.prompt_template.format(question=question)

def parse_gold(answer_text: str) -> Optional[str]:
    m = re.search(r"####\s*(-?\d+(?:\.\d+)?)", answer_text)
    if m: return m.group(1).strip()
    nums = re.findall(r"-?\d+(?:\.\d+)?", answer_text)
    return nums[-1].strip() if nums else None

def parse_pred(text: str) -> Optional[str]:
    m = re.search(r"\[\s*(-?\d+(?:\.\d+)?)\s*\]", text)
    if m: return m.group(1).strip()
    nums = re.findall(r"-?\d+(?:\.\d+)?", text)
    return nums[-1].strip() if nums else None

print("Loading GSM8K…")
ds_train_full = load_dataset("openai/gsm8k", "main", split="train")
ds_test       = load_dataset("openai/gsm8k", "main", split="test")

if cfg.val_rows is None:
    val_rows = min(200, len(ds_train_full))
else:
    val_rows = min(cfg.val_rows, len(ds_train_full))

ds_val   = ds_train_full.select(range(val_rows))
ds_train = ds_train_full.select(range(val_rows, len(ds_train_full)))

print(f"Splits: {len(ds_train)} train | {len(ds_val)} val | {len(ds_test)} test")

## Load Models

Key difference: we now load **multiple teacher models** and store them in a list with their tokenizers.

In [ ]:
# --------------------------
# Tokenizers & Models
# --------------------------
def load_tokenizer(model_id: str):
    tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    if tok.pad_token is None and tok.eos_token is not None:
        tok.pad_token = tok.eos_token
    tok.padding_side = "left"  # decoder-only: left padding
    return tok

def make_lora_student(model_id: str) -> torch.nn.Module:
    base = AutoModelForCausalLM.from_pretrained(
        model_id, torch_dtype=torch.bfloat16, device_map="auto"
    )
    base.config.use_cache = False  # off for training
    lora_cfg = LoraConfig(
        r=16, lora_alpha=32, lora_dropout=0.05, bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    )
    return get_peft_model(base, lora_cfg)

def load_teacher(model_id: str) -> torch.nn.Module:
    m = AutoModelForCausalLM.from_pretrained(
        model_id, torch_dtype=torch.bfloat16, device_map="auto"
    ).eval()
    for p in m.parameters():
        p.requires_grad_(False)
    return m

# Load student
print(f"Loading student: {cfg.student_id}")
tok_s = load_tokenizer(cfg.student_id)

# Load all teachers
print(f"\nLoading {len(cfg.teachers)} teachers...")
teachers = []
teacher_tokenizers = []
for i, t_cfg in enumerate(cfg.teachers, 1):
    print(f"  [{i}/{len(cfg.teachers)}] {t_cfg.model_id}")
    teacher_model = load_teacher(t_cfg.model_id)
    teacher_tok = load_tokenizer(t_cfg.model_id)
    teachers.append(teacher_model)
    teacher_tokenizers.append(teacher_tok)
    print(f"       Loaded. Weight: {t_cfg.weight:.3f}")

print(f"\nAll models loaded. Student padding: {tok_s.padding_side}")

## Baseline Evaluation

Let's evaluate all models (student + all teachers) before training to establish baselines.

In [ ]:
# --------------------------
# Evaluation utils
# --------------------------
def _encode_cuda(tokenizer, texts: List[str], max_length=2048) -> Dict[str, torch.Tensor]:
    enc = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    return {k: v.to("cuda") for k, v in enc.items()}

@torch.no_grad()
def evaluate(model, tokenizer, dataset, *, num_examples: Optional[int] = None,
             temperature: float = 0.0, max_new_tokens: int = 256,
             batch_size: int = 32, desc: str = "Eval") -> float:
    """Exact-match accuracy against bracketed numeric answer."""
    n = len(dataset) if num_examples is None else min(num_examples, len(dataset))
    rows = dataset.select(range(n))
    correct = 0

    was_cache = getattr(model.config, "use_cache", True)
    model.eval(); model.config.use_cache = True

    for i in tqdm(range(0, n, batch_size), desc=desc):
        batch = rows.select(range(i, min(i + batch_size, n)))
        prompts = [render_prompt(ex["question"]) for ex in batch]
        enc = _encode_cuda(tokenizer, prompts)

        gen_kwargs = dict(
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True
        )
        if temperature and temperature > 0.0:
            gen_kwargs.update(do_sample=True, temperature=temperature, top_p=0.9)
        else:
            gen_kwargs.update(do_sample=False)

        outs = model.generate(**enc, **gen_kwargs)
        txts = tokenizer.batch_decode(outs, skip_special_tokens=True)

        for ex, text in zip(batch, txts):
            pred = parse_pred(text) or ""
            gold = parse_gold(ex["answer"]) or ""
            correct += int(pred == gold)

    model.config.use_cache = was_cache
    return correct / max(n, 1)

@torch.no_grad()
def preview(model, tokenizer, dataset, k=2, temperature=0.0, max_new_tokens=256):
    model.eval(); model.config.use_cache = True
    rows = dataset.select(range(min(k, len(dataset))))
    for ex in rows:
        prompt = render_prompt(ex["question"])
        enc = tokenizer([prompt], return_tensors="pt").to(model.device)
        gen_kwargs = dict(max_new_tokens=max_new_tokens, use_cache=True,
                          pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id)
        if temperature and temperature > 0.0:
            gen_kwargs.update(do_sample=True, temperature=temperature, top_p=0.9)
        out = model.generate(**enc, **gen_kwargs)
        text = tokenizer.decode(out[0], skip_special_tokens=True)
        print("="*80); print(prompt); print("-"*80); print(text)
        print("-"*80, f"\nParsed: [{parse_pred(text)}] | Gold: [{parse_gold(ex['answer'])}]")

# --------------------------
# Baselines (before training)
# --------------------------
print("\n== Computing Baselines ==")
student_for_eval = make_lora_student(cfg.student_id)

print("\nPreview (student, greedy):")
preview(student_for_eval, tok_s, ds_test, k=1, temperature=0.0, max_new_tokens=cfg.max_new_tokens)

print("\nEvaluating student...")
baseline_student_em = evaluate(student_for_eval, tok_s, ds_test,
                               temperature=cfg.eval_temperature, max_new_tokens=cfg.max_new_tokens,
                               batch_size=128, desc="Student baseline (test)")
print(f"Student (0.6B base) EM: {baseline_student_em:.4f}")

# Evaluate all teachers
teacher_baselines = []
for i, (teacher, tok_t, t_cfg) in enumerate(zip(teachers, teacher_tokenizers, cfg.teachers), 1):
    print(f"\nEvaluating teacher {i}/{len(teachers)}: {t_cfg.model_id}")
    em = evaluate(teacher, tok_t, ds_test,
                  temperature=cfg.eval_temperature, max_new_tokens=cfg.max_new_tokens,
                  batch_size=64, desc=f"Teacher {i} baseline")
    teacher_baselines.append(em)
    print(f"Teacher {i} EM: {em:.4f}")

print("\n=== Baseline Summary ===")
print(f"Student: {baseline_student_em:.4f}")
for i, (em, t_cfg) in enumerate(zip(teacher_baselines, cfg.teachers), 1):
    print(f"Teacher {i} ({t_cfg.model_id.split('/')[-1]}): {em:.4f} (weight: {t_cfg.weight:.3f})")

baselines_dict = {
    "student": baseline_student_em,
    **{f"teacher_{i}": em for i, em in enumerate(teacher_baselines, 1)}
}
with open(os.path.join(cfg.run_root, "baselines.json"), "w") as f:
    json.dump(baselines_dict, f, indent=2)

# Free the temporary student
del student_for_eval; torch.cuda.empty_cache()

## Multi-Teacher Training Utilities

The key innovation is in the `multi_teacher_logp` function, which:
1. Computes log probabilities from each teacher
2. Combines them using weighted averaging (or other aggregation strategies)
3. Returns a single aggregated teacher signal for the student

In [ ]:
# --------------------------
# Multi-Teacher OPD utilities
# --------------------------
def mask_before_first_eos(next_ids: torch.Tensor, eos_id: int) -> torch.Tensor:
    """Mask tokens strictly before the first EOS in each sequence."""
    is_eos = (next_ids == eos_id)
    csum = is_eos.cumsum(dim=1)
    return csum.eq(0)

def student_logp_batched(student, pad_id, full_ids, next_ids, T, micro_bsz):
    """Return student log p(a_t) for the last T tokens (with grad)."""
    outs = []
    for s in range(0, full_ids.size(0), micro_bsz):
        chunk = full_ids[s:s+micro_bsz]; nxt = next_ids[s:s+micro_bsz]
        out = student(input_ids=chunk[:, :-1],
                      attention_mask=(chunk[:, :-1] != pad_id))
        logits = out.logits[:, -T:, :]
        logp = F.log_softmax(logits, dim=-1).gather(-1, nxt.unsqueeze(-1)).squeeze(-1)
        outs.append(logp)
        del out, logits
    return torch.cat(outs, dim=0)

# ---------- Cross-tokenizer teacher scoring ----------
def _decode_token_str(tokenizer, token_id: int) -> str:
    return tokenizer.decode([int(token_id)],
                            skip_special_tokens=False,
                            clean_up_tokenization_spaces=False)

def _encode_text_ids(tokenizer, text: str):
    return tokenizer(text,
                     add_special_tokens=False,
                     return_tensors="pt").input_ids[0].tolist()

@torch.no_grad()
def teacher_logp_grouped_by_student_tokens(
    teacher, tok_teacher, tok_student, prompts: List[str], next_ids: torch.Tensor, max_len: Optional[int] = None
):
    """
    For each sample b and student step t:
      1) Decode the student's token id next_ids[b,t] → text piece
      2) Tokenize that text with the teacher tokenizer (may become multiple tokens)
      3) Sum teacher log-probs over that group
    Returns: Tensor [B, T] on CUDA with per-student-step teacher log-probs.
    """
    device = teacher.device if hasattr(teacher, "device") else "cuda"
    B, T = next_ids.shape
    out = torch.zeros((B, T), device=device, dtype=torch.float32)

    if max_len is None:
        max_len = int(getattr(teacher.config, "max_position_embeddings", 2048))

    for b in range(B):
        prompt_text = prompts[b]
        ctx_ids = _encode_text_ids(tok_teacher, prompt_text)

        groups = []
        for t in range(T):
            s_tok_id = int(next_ids[b, t].item())
            piece = _decode_token_str(tok_student, s_tok_id)
            ids_t = _encode_text_ids(tok_teacher, piece)
            groups.append(ids_t)

        flat_gen = [tid for g in groups for tid in g]
        if len(flat_gen) == 0:
            continue

        # Respect the teacher's context length
        total = len(ctx_ids) + len(flat_gen)
        if total > max_len:
            overflow = total - max_len
            ctx_ids = ctx_ids[overflow:]

        full = ctx_ids + flat_gen
        if len(full) < 2:
            continue

        input_ids = torch.tensor(full[:-1], device=device).unsqueeze(0)
        labels    = torch.tensor(full[1:],  device=device).unsqueeze(0)
        attn_mask = torch.ones_like(input_ids, device=device)

        outputs = teacher(input_ids=input_ids, attention_mask=attn_mask)
        logprobs = F.log_softmax(outputs.logits, dim=-1)
        tok_lp = logprobs.gather(-1, labels.unsqueeze(-1)).squeeze(-1)[0]

        start = max(len(ctx_ids) - 1, 0)
        gen_lp = tok_lp[start : start + len(flat_gen)]

        # Sum back per student step
        off = 0
        for t, g in enumerate(groups):
            k = len(g)
            if k > 0:
                out[b, t] = gen_lp[off : off + k].sum()
            off += k

    return out

@torch.no_grad()
def multi_teacher_logp(
    teachers: List[torch.nn.Module],
    teacher_toks: List[AutoTokenizer],
    teacher_weights: List[float],
    tok_student: AutoTokenizer,
    prompts: List[str],
    next_ids: torch.Tensor,
    aggregation: str = "weighted_avg"  # or "max", "mixture"
) -> torch.Tensor:
    """
    Compute aggregated teacher log-probs from multiple teachers.
    
    Args:
        teachers: List of teacher models
        teacher_toks: List of teacher tokenizers
        teacher_weights: Normalized weights for each teacher
        tok_student: Student tokenizer
        prompts: List of prompt strings
        next_ids: Student-generated token IDs [B, T]
        aggregation: How to combine teacher signals
            - "weighted_avg": Weighted average of log-probs (default)
            - "max": Take maximum log-prob at each position
            - "mixture": Log of weighted probability mixture
    
    Returns:
        Aggregated log-probs [B, T]
    """
    B, T = next_ids.shape
    device = next_ids.device
    
    # Collect log-probs from all teachers
    all_logps = []
    for teacher, tok_t in zip(teachers, teacher_toks):
        logp_t = teacher_logp_grouped_by_student_tokens(
            teacher=teacher,
            tok_teacher=tok_t,
            tok_student=tok_student,
            prompts=prompts,
            next_ids=next_ids,
            max_len=getattr(teacher.config, "max_position_embeddings", 2048)
        )
        all_logps.append(logp_t)
    
    # Stack: [num_teachers, B, T]
    stacked = torch.stack(all_logps, dim=0)
    
    # Aggregate
    if aggregation == "weighted_avg":
        # Weighted average in log space (not exact, but simple)
        weights = torch.tensor(teacher_weights, device=device).view(-1, 1, 1)
        result = (stacked * weights).sum(dim=0)
    elif aggregation == "max":
        # Take the maximum log-prob at each position
        result = stacked.max(dim=0)[0]
    elif aggregation == "mixture":
        # Proper mixture: log(sum_i w_i * exp(logp_i))
        weights = torch.tensor(teacher_weights, device=device).view(-1, 1, 1)
        # Use logsumexp for numerical stability
        # log(sum w_i exp(logp_i)) = log(sum exp(log(w_i) + logp_i))
        log_weights = torch.log(weights)
        result = torch.logsumexp(log_weights + stacked, dim=0)
    else:
        raise ValueError(f"Unknown aggregation: {aggregation}")
    
    return result

class LiveTable:
    def __init__(self, title: str = "Training metrics", max_rows: int = 200):
        self.title = title
        self.max_rows = max_rows
        self.rows = []
        empty = pd.DataFrame(columns=["step","loss","loss_ema","revkl","revkl_ema","tokens","val_em"])
        self.handle = display(self._styled(empty), display_id=True)

    def _styled(self, df: pd.DataFrame):
        styler = (
            df.style
              .set_caption(self.title)
              .format({
                  "loss": "{:.4f}",
                  "loss_ema": "{:.4f}",
                  "revkl": "{:.4f}",
                  "revkl_ema": "{:.4f}",
                  "val_em": (lambda v: "" if pd.isna(v) else f"{v:.3f}"),
              })
        )
        try:
            styler = styler.hide(axis="index")
            return styler
        except Exception:
            pass
        return styler.set_table_styles([
            {"selector": "th.row_heading", "props": [("display", "none")]},
            {"selector": "th.blank",       "props": [("display", "none")]},
        ])

    def update(self, *, step, loss, loss_ema, revkl, revkl_ema, tokens, val_em=None):
        self.rows.append(dict(
            step=int(step),
            loss=float(loss),
            loss_ema=(None if loss_ema is None else float(loss_ema)),
            revkl=float(revkl),
            revkl_ema=(None if revkl_ema is None else float(revkl_ema)),
            tokens=int(tokens),
            val_em=(None if val_em is None else float(val_em)),
        ))
        rows = self.rows[-self.max_rows:]
        df = pd.DataFrame(rows, columns=["step","loss","loss_ema","revkl","revkl_ema","tokens","val_em"])
        self.handle.update(self._styled(df))

## Training Metrics Explanation

| Column        | Meaning                                                                                                                                                                                                         | How to interpret                                                                      |
| :------------ | :-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------ |
| **step**      | The current training iteration (out of the total configured `cfg.steps`). Each step processes one batch of sampled prompts.                                                                                     | Training progresses along this axis.                                                  |
| **loss**      | The instantaneous batch loss (reverse-KL–style policy-gradient objective). Negative values are expected because we optimize `-E[(log p_T − log p_S) · log p_S]`; good updates drive this lower (more negative). | Lower (more negative) → better alignment with teachers on that batch.                 |
| **loss_ema**  | Exponential moving average (EMA) of the loss across steps. Smooths noise to show trend.                                                                                                                         | Downward trend = overall improvement; steadiness = convergence.                       |
| **revkl**     | The *reverse-KL divergence estimate* for this batch: roughly E[ log p_S − log p_T ]. It measures how far the student's distribution is from the **aggregated teachers'** distribution.                          | Smaller (approaching 0) → student policy is closer to teacher ensemble.               |
| **revkl_ema** | EMA-smoothed version of reverse-KL.                                                                                                                                                                             | Should decrease steadily if multi-teacher distillation is working.                    |
| **tokens**    | Cumulative count of *valid graded tokens* seen so far — all pre-EOS generated tokens used for loss computation.                                                                                                 | Measures total learning signal processed; roughly proportional to compute/throughput. |
| **val_em**    | Validation exact-match accuracy (fraction of GSM8K val examples where the parsed numeric answer matches gold). Evaluated every `cfg.val_every` steps.                                                           | Direct measure of task performance; higher = better reasoning accuracy.               |

In [ ]:
# --------------------------
# Multi-Teacher Training Loop
# --------------------------
def ema(prev, new, beta):
    return new if prev is None else (beta * prev + (1 - beta) * new)

def run_multi_teacher_training(run_dir: str, aggregation: str = "weighted_avg"):
    """
    Main multi-teacher on-policy distillation training loop.
    
    Args:
        run_dir: Directory to save outputs
        aggregation: How to combine teacher signals ("weighted_avg", "max", "mixture")
    """
    os.makedirs(run_dir, exist_ok=True)

    # Fresh student (LoRA adapters)
    student = make_lora_student(cfg.student_id)
    optimizer = torch.optim.AdamW(student.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)

    prompts_all = [render_prompt(x["question"]) for x in ds_train]
    ema_loss = None
    ema_revkl = None
    tokens_graded_cum = 0
    logs = []

    table = LiveTable(title=f"Multi-Teacher OPD ({len(cfg.teachers)} teachers, {aggregation})")

    # Extract normalized weights
    teacher_weights = [t.weight for t in cfg.teachers]

    pbar = tqdm(range(cfg.steps), desc=f"Multi-Teacher OPD [{os.path.basename(run_dir)}]")
    for step in pbar:
        # Deterministic batch selection per step
        rng = np.random.default_rng(SEED + step)
        idxs = rng.choice(len(prompts_all), size=cfg.batch_prompts, replace=False)
        prompts = [prompts_all[i] for i in idxs]
        prompts_rep = sum(([p] * cfg.samples_per_prompt for p in prompts), [])
        enc = tok_s(prompts_rep, padding=True, truncation=True, max_length=2048, return_tensors="pt").to("cuda")

        # 1) Student rollouts (no grad) to get sequences and step count
        with torch.no_grad():
            gen_out = student.generate(
                **enc,
                do_sample=True, temperature=cfg.train_temperature, top_p=0.9,
                max_new_tokens=cfg.max_new_tokens,
                eos_token_id=tok_s.eos_token_id, pad_token_id=tok_s.pad_token_id,
                return_dict_in_generate=True, output_scores=True
            )
            seqs = gen_out.sequences
            scores_list = list(gen_out.scores)
            T = len(scores_list)
            next_ids = seqs[:, -T:]
            valid_mask = mask_before_first_eos(next_ids, eos_id=tok_s.eos_token_id).float()

        # 2) Student log-probs with grad
        student.train(); student.config.use_cache = False
        logp_s = student_logp_batched(
            student, tok_s.pad_token_id, seqs, next_ids, T, cfg.student_mb
        )

        # 3) MULTI-TEACHER aggregated log-probs (no grad)
        logp_t = multi_teacher_logp(
            teachers=teachers,
            teacher_toks=teacher_tokenizers,
            teacher_weights=teacher_weights,
            tok_student=tok_s,
            prompts=prompts_rep,
            next_ids=next_ids,
            aggregation=aggregation
        )

        # 4) Reverse-KL-style policy gradient
        adv = (logp_t - logp_s).clamp(-5, 5)
        denom = valid_mask.sum().clamp_min(1.0)
        loss = - ((valid_mask * adv.detach()) * logp_s).sum() / denom

        loss.backward()
        torch.nn.utils.clip_grad_norm_(student.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)

        with torch.no_grad():
            rev_kl = ((logp_s - logp_t) * valid_mask).sum().item() / float(denom)
            tokens_graded_cum += int(denom.item())
            ema_loss = ema(ema_loss, float(loss.item()), cfg.ema_momentum)
            ema_revkl = ema(ema_revkl, float(rev_kl), cfg.ema_momentum)

        # Periodic validation EM (greedy)
        val_em = None
        if (step % cfg.val_every == 0) or (step == cfg.steps - 1):
            student.eval(); student.config.use_cache = True
            val_em = evaluate(student, tok_s, ds_val,
                              num_examples=min(cfg.val_sample_n, len(ds_val)),
                              temperature=0.0, max_new_tokens=cfg.max_new_tokens,
                              batch_size=32, desc="VAL EM")
            student.train(); student.config.use_cache = False

        row = dict(
            step=int(step),
            train_loss=float(loss.item()),
            train_loss_ema=float(ema_loss) if ema_loss is not None else None,
            train_revkl=float(rev_kl),
            train_revkl_ema=float(ema_revkl) if ema_revkl is not None else None,
            tokens_graded=int(tokens_graded_cum),
            **({"val_em": float(val_em)} if val_em is not None else {})
        )
        logs.append(row)

        if (step % cfg.log_every == 0) or (val_em is not None):
            table.update(
                step=row["step"],
                loss=row["train_loss"],
                loss_ema=row["train_loss_ema"],
                revkl=row["train_revkl"],
                revkl_ema=row["train_revkl_ema"],
                tokens=row["tokens_graded"],
                val_em=row.get("val_em", None),
            )

        postfix = {
            "loss": f"{loss.item():.3f}",
            "ema": f"{(ema_loss if ema_loss is not None else loss.item()):.3f}",
            "rkl": f"{rev_kl:.3f}",
            "toks": tokens_graded_cum
        }
        if val_em is not None:
            postfix["val"] = f"{val_em:.3f}"
        pbar.set_postfix(**postfix)

        del scores_list
        torch.cuda.empty_cache()

    # Save logs
    try:
        pd.DataFrame(logs).to_csv(os.path.join(run_dir, "train_logs.csv"), index=False)
    except Exception:
        with open(os.path.join(run_dir, "train_logs.jsonl"), "w") as f:
            for r in logs:
                f.write(json.dumps(r) + "\n")

    # Final test EM (greedy)
    student.eval(); student.config.use_cache = True
    test_em = evaluate(student, tok_s, ds_test,
                       temperature=0.0, max_new_tokens=cfg.max_new_tokens,
                       batch_size=64, desc=f"Test EM [{os.path.basename(run_dir)}]")

    # Save adapters and summary
    save_dir = os.path.join(run_dir, "adapters_lora")
    os.makedirs(save_dir, exist_ok=True)
    student.save_pretrained(save_dir)

    summary = dict(
        num_teachers=len(cfg.teachers),
        aggregation=aggregation,
        teacher_weights=[t.weight for t in cfg.teachers],
        steps=cfg.steps,
        batch_prompts=cfg.batch_prompts,
        samples_per_prompt=cfg.samples_per_prompt,
        max_new_tokens=cfg.max_new_tokens,
        train_tokens_graded=tokens_graded_cum,
        test_em=float(test_em)
    )
    with open(os.path.join(run_dir, "summary.json"), "w") as f:
        json.dump(summary, f, indent=2)

    # Free GPU
    del student; torch.cuda.empty_cache()
    return summary, logs

## Run Multi-Teacher Training

Now let's train the student with all teachers!

In [ ]:
# --------------------------
# Run training
# --------------------------
print("\n== Multi-Teacher Training ==")
print(f"Aggregation strategy: weighted_avg")
print(f"Number of teachers: {len(cfg.teachers)}")

run_dir = os.path.join(cfg.run_root, "multi_teacher_opd")
summary, _ = run_multi_teacher_training(run_dir, aggregation="weighted_avg")

print("\n== Final Results ==")
final_report = dict(
    env=dict(
        python=sys.version,
        torch=torch.__version__,
        cuda=torch.version.cuda if torch.cuda.is_available() else "cpu",
        device=torch.cuda.get_device_name(0) if torch.cuda.is_available() else "cpu",
    ),
    config=dict(
        student=cfg.student_id,
        teachers=[t.model_id for t in cfg.teachers],
        teacher_weights=[t.weight for t in cfg.teachers],
        prompt_template=cfg.prompt_template,
    ),
    baselines=baselines_dict,
    final=summary,
    improvement=float(summary["test_em"] - baseline_student_em)
)

print(json.dumps(final_report, indent=2))

print(f"\n=== Summary ===")
print(f"Student baseline: {baseline_student_em:.4f}")
print(f"After multi-teacher OPD: {summary['test_em']:.4f}")
print(f"Improvement: {final_report['improvement']:.4f}")
print(f"\nArtifacts saved to: {cfg.run_root}")

## Comparison: Different Aggregation Strategies (Optional)

You can experiment with different ways to combine teacher signals:

In [ ]:
# Optional: Compare aggregation strategies
# Uncomment to run multiple experiments

# aggregation_strategies = ["weighted_avg", "max", "mixture"]
# results = {}

# for agg in aggregation_strategies:
#     print(f"\n{'='*80}")
#     print(f"Training with aggregation: {agg}")
#     print(f"{'='*80}")
#     
#     run_dir = os.path.join(cfg.run_root, f"multi_teacher_{agg}")
#     summary, _ = run_multi_teacher_training(run_dir, aggregation=agg)
#     results[agg] = summary
#     
#     print(f"\nTest EM ({agg}): {summary['test_em']:.4f}")
#     torch.cuda.empty_cache()

# # Compare results
# print("\n" + "="*80)
# print("COMPARISON: Aggregation Strategies")
# print("="*80)
# print(f"Student baseline: {baseline_student_em:.4f}")
# for agg, summary in results.items():
#     improvement = summary['test_em'] - baseline_student_em
#     print(f"{agg:15s}: {summary['test_em']:.4f} (+{improvement:.4f})")

## Key Takeaways

### Multi-Teacher vs Single-Teacher OPD

**Advantages:**
1. **Ensemble benefits**: Multiple teachers provide more robust guidance
2. **Diverse knowledge**: Different teachers may excel at different aspects
3. **Flexibility**: Can weight teachers by performance or other criteria

**Considerations:**
1. **Compute cost**: Must score student rollouts with multiple teachers (linear in number of teachers)
2. **Memory**: All teachers must fit in GPU memory (though can be on separate GPUs)
3. **Aggregation strategy**: Choice of how to combine teachers affects results

### Aggregation Strategies

1. **Weighted Average** (default): Simple, interpretable, works well in practice
2. **Max**: Takes best teacher signal at each step, but ignores weights
3. **Mixture**: Proper probabilistic mixture, theoretically sound but more complex

### Extension Ideas

1. **Dynamic weighting**: Adjust teacher weights during training based on agreement
2. **Specialized teachers**: Use different teachers for different data distributions
3. **Curriculum**: Start with easier teachers, gradually add harder ones
4. **Teacher selection**: Per-batch teacher selection based on uncertainty

### Production Considerations

For the production implementation shown in the tinker-cookbook:
- Uses much larger models (8B student, 32B and 235B teachers)
- Separate datasets for each teacher (DeepMath, Tulu3)
- More sophisticated infrastructure (distributed training, checkpointing)
- Consider cost vs. benefit: Does the improvement justify 2-3x compute?